### Resolución test Damián Mateos

In [1]:
# Se importan los módulos a utilizar

import numpy as np
import pandas as pd
from os import path

import warnings
warnings.filterwarnings('ignore')

In [2]:
pathing = path.join('Sample test file - Sheet1.csv')
dataset = pd.read_csv(pathing)

display(dataset)

,First Name,Last Name,Street,Zip,City,Type,Last Check-In Date,Job,Phone,Company
0,Federico,García,Recogidas Street,18805,Granada,U,14/01/2018,Dentist,958 419182,Badger Maps
1,Ángel,Ganivet,Puerta Real Avenue,18805,Granada,U,06/02/2018,Salesman,917891232,Badger Maps
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,André,Citröen,Gran Via Road,18805,Granada,D,1/3/2018,Postman,971 659 123,Badger Maps
4,Anselmo,Ortega,King St,DE1 3LD,Almería,A,12/11/2017,Policeman,933 12 41 59,Wood SA
5,Bjorn,Ostberg,"27, Queen St",DE1 3LD,Almería,U,24/04/2018,Actor,93 1 31 21 29,Wood SA
6,Markus,Dupont,"102, Fake Street",18005,Granada,A,NaN,Musician,912234223,Fabric SA
7,Jake,Robins,Street Road,NaN,Granada,D,11/03/2018,Assistant,1111111111,Fabric SA
8,Lola,Rivers,NaN,18005,Granada,A,21/02/2018,CEO,958963229,Fabric SA
9,Paul,Hudson,Leicester Square,18005,Granada,D,24/01/2018,Salesman,953912345,Fabric SA


In [3]:
dataset.drop_duplicates(inplace = True)

display(dataset.shape)

(10, 10)

Se comprueba que no existen datos duplicados en el dataset, pues sigue teniendo las mismas dimensiones. A continuación se van a seleccionar los atributos o campos relevantes del DataFrame, que son los mencionados en las instrucciones.

In [4]:
data = dataset[['First Name', 'Last Name', 'Street', 'Zip', 'City', 'Last Check-In Date', 'Company']]

data.describe()

,First Name,Last Name,Street,Zip,City,Last Check-In Date,Company
count,9,9,8,8,9,8,9
unique,9,9,8,3,2,8,3
top,Federico,Hudson,Leicester Square,18805,Granada,12/11/2017,Fabric SA
freq,1,1,1,3,7,1,4


Observando que hay 10 instancias de datos y viendo los valores no nulos en cada columna, se comprueba que todas ellas tienen al menos un NaN. Se procede al tratamiento de valores nulos, para lo que se optará por eliminar las filas donde no haya al menos 2 datos no nulos. 

In [5]:
rows_nan = data[data.isnull().any(1)]

display(rows_nan)

,First Name,Last Name,Street,Zip,City,Last Check-In Date,Company
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Markus,Dupont,"102, Fake Street",18005,Granada,NaN,Fabric SA
7,Jake,Robins,Street Road,NaN,Granada,11/03/2018,Fabric SA
8,Lola,Rivers,NaN,18005,Granada,21/02/2018,Fabric SA


Estas son las muestras donde existen valores nulos. La 2 se eliminará al no superar el umbral de dos datos no nulos, mientras que el resto se sustituirán por valores no nulos.

Antes de eso se pasan las fechas a formato datetime para que no sean strings y se pueda trabajar con ellas.

In [6]:
data['Last Check-In Date'] = pd.to_datetime(data['Last Check-In Date'], format = '%d/%m/%Y')

Se opta por sustituir la fecha por la media de ellas, que ya se puede hacer por estar en formato datetime. El resto de datos se sustituyen por el string 'unknown' salvo el Zip que se incluye un -1. Así se muestran diferentes formas de dar valor a los datos nulos, se ha elegido la media para la fecha por no afectar esta al cliente más antiguo o reciente.

In [7]:
data.dropna(thresh = 2, inplace = True) # filas que no tengan al menos dos datos no nulos se eliminan
mean_date = data['Last Check-In Date'].mean() # media para los datos que no tengan fecha 

filling_values = {'First Name': 'unknown', 'Last Name': 'unknown', 'Street': 'unknown', 'Zip': -1, 'City': 'unknown', 
                  'Last Check-In Date': mean_date, 'Company': 'unknown'}

data.fillna(value = filling_values, inplace = True)
data['Last Check-In Date'] = data['Last Check-In Date'].dt.date # para no mostrar horas, minutos y segundos

display(data)

,First Name,Last Name,Street,Zip,City,Last Check-In Date,Company
0,Federico,García,Recogidas Street,18805,Granada,2018-01-14,Badger Maps
1,Ángel,Ganivet,Puerta Real Avenue,18805,Granada,2018-02-06,Badger Maps
3,André,Citröen,Gran Via Road,18805,Granada,2018-03-01,Badger Maps
4,Anselmo,Ortega,King St,DE1 3LD,Almería,2017-11-12,Wood SA
5,Bjorn,Ostberg,"27, Queen St",DE1 3LD,Almería,2018-04-24,Wood SA
6,Markus,Dupont,"102, Fake Street",18005,Granada,2018-02-09,Fabric SA
7,Jake,Robins,Street Road,-1,Granada,2018-03-11,Fabric SA
8,Lola,Rivers,unknown,18005,Granada,2018-02-21,Fabric SA
9,Paul,Hudson,Leicester Square,18005,Granada,2018-01-24,Fabric SA


Como el nombre de cada cliente interesa completo, van a unirse las dos primeras columnas.

In [8]:
full_name = data['First Name'].astype(str) + ' ' + data['Last Name'].astype(str)
data.insert(0, 'Full Name', full_name)
data.drop(['First Name', 'Last Name'], inplace = True, axis = 1)

display(data.sample(2)) # sampleo de dos muestras para ver el resultado

,Full Name,Street,Zip,City,Last Check-In Date,Company
8,Lola Rivers,unknown,18005,Granada,2018-02-21,Fabric SA
5,Bjorn Ostberg,"27, Queen St",DE1 3LD,Almería,2018-04-24,Wood SA


Finalmente, los datos procesados quedan de la siguiente forma:

In [9]:
display(data)

,Full Name,Street,Zip,City,Last Check-In Date,Company
0,Federico García,Recogidas Street,18805,Granada,2018-01-14,Badger Maps
1,Ángel Ganivet,Puerta Real Avenue,18805,Granada,2018-02-06,Badger Maps
3,André Citröen,Gran Via Road,18805,Granada,2018-03-01,Badger Maps
4,Anselmo Ortega,King St,DE1 3LD,Almería,2017-11-12,Wood SA
5,Bjorn Ostberg,"27, Queen St",DE1 3LD,Almería,2018-04-24,Wood SA
6,Markus Dupont,"102, Fake Street",18005,Granada,2018-02-09,Fabric SA
7,Jake Robins,Street Road,-1,Granada,2018-03-11,Fabric SA
8,Lola Rivers,unknown,18005,Granada,2018-02-21,Fabric SA
9,Paul Hudson,Leicester Square,18005,Granada,2018-01-24,Fabric SA


Sobre las respuestas requeridas en cuanto a los clientes, se muestran a continuación:

In [10]:
ordered_data = data.sort_values(by = 'Last Check-In Date', ascending = False, inplace = False)
#display(ordered_data)

ancient_customer = ordered_data.iloc[-1, 0]
recent_customer = ordered_data.iloc[0, 0]

print(f'El cliente con el check-in-date más antiguo es {ancient_customer}.\n')
print(f'El cliente con el check-in-date más reciente es {recent_customer}.')

El cliente con el check-in-date más antiguo es Anselmo Ortega.

El cliente con el check-in-date más reciente es Bjorn Ostberg.


Por último, se obtiene una lista de los clientes ordenados alfabéticamente:

In [11]:
customers_array = data['Full Name'].unique()
customers_list = customers_array.tolist()
customers_list.sort(key = lambda x: x.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u'))
print(f'Los clientes, por orden alfabético son:\n\n{customers_list}')

Los clientes, por orden alfabético son:

['André Citröen', 'Ángel Ganivet', 'Anselmo Ortega', 'Bjorn Ostberg', 'Federico García', 'Jake Robins', 'Lola Rivers', 'Markus Dupont', 'Paul Hudson']


Destacar que se comprobó que Ángel se quedaba el último por el orden en el alfabeto ACSII de Python, por lo que se reemplazaron las vocales con tilde por aquellas sin tilde solamente para ordenarlas. También se pasaron todas a minúsculas por comodidad y evitar problemas si algún nombre estuviese en minúscula.